This notebook is provided as basic demonstration of use of the pmapUtilities R package, developed by Luke C. Mullany, PhD MS MHS, and available at https://github.com/lmullany/pmapUtilities.git and can be installed using `devtools::install_github("lmullany/pmapUtilities")`

- **Platform Tool** : Jupyter/RStudio Crunchr Compute Containers
- **Programming Language**: R (>=3.6)
- **Author(s)** : Luke C. Mullany
- **Last Updated** : September 12, 2022

## Basic use of pmapUtilities R package

#### Clear workspace and load libraries

In [1]:
rm(list=ls())
library(pmapUtilities)

#### Get information on available databases
Use the `get_database_names()` function, passing your username (JHED ID). This will return a three column tibble with columns `name`, `database_id`, and `create_date`. You can optionally pass an argumen to the `pattern` parameter, to filter the results to a specific subset of database names

In [2]:
dbs = get_database_names(username="lmullan1")

Enter Password for lmullan1:  ·········


In [3]:
tail(dbs,3)

name,database_id,create_date
<chr>,<int>,<dttm>
Kidney_Menez_IRB00304696_Scratch,305,2022-09-08 16:37:58
Kidney_Menez_IRB00304696_Projection,306,2022-09-08 16:38:09
NCCU_OMOP,307,2022-09-08 16:42:35


Here we limit our search to just databases where name starts with `"PatientSafety"`

In [4]:
dbs = get_database_names(username = "lmullan1", pattern="^PatientSafety")

Enter Password for lmullan1:  ·········


In [5]:
dbs

name,database_id,create_date
<chr>,<int>,<dttm>
PatientSafetyQualityWSP_Scratch,123,2021-03-02 07:38:28
PatientSafetyQualityVTE_Scratch,182,2021-07-27 09:57:07
PatientSafetyQualityMA_Projection,219,2022-01-04 16:25:42
PatientSafetyQualityMA_Scratch,220,2022-01-04 16:25:56
PatientSafetyQualityWSP_Projection,221,2022-03-16 08:42:08
PatientSafetyQuality_JHM_Keystone_Scratch,269,2022-05-06 10:54:07
PatientSafetyQuality_JHM_Keystone_Projection,270,2022-05-06 10:54:20
PatientSafetyQualityVTE_Projection,299,2022-08-19 08:55:42


#### Generate a connection to the database
Here, we use the `get_sql_connection()` function, passing a database name, and a username (JHED ID). You can assign this object to `default_engine` (or any other name). The advantage of using `default_engine` is that all subsequent `pmapUtilities` functions that require a connection object will look by default in the global environment for `default_engine`

In [6]:
default_engine = get_sql_connection(dbname = "CAMP_PMCoE_Projection",username = "lmullan1")

Enter Password for lmullan1:  ·········


#### Show all the tables in this database

In [7]:
list_tables()

table
<chr>
patients
encounters
labs
meds
problemlist
procedures
symptoms
vitals_BP
vitals_height


#### Call the same function, but this time ask for dimensions (`show_dimensions = TRUE`)

In [8]:
list_tables(show_dimensions = TRUE, exact=T)

table,rows,cols
<chr>,<dbl>,<dbl>
patients,60676,5
encounters,753484,4
labs,3509868,12
meds,631022,10
problemlist,781379,4
procedures,5550672,6
symptoms,1967436,5
vitals_BP,1061684,7
vitals_height,305612,7


#### List columns for a particular table

In [10]:
list_columns("encounters")

[1] "osler_id"       "encounter_id"   "encounter_type" "encounter_date"

#### Get the number of rows and columns for just one table (rather than all tables, see above)

In [11]:
get_table_dim("encounters", exact = T)

rows   cols 
753484      4

#### Return a lazy handle to the table
Note that this feature is using dplyr/dbplyr under the hood, which enables us to translate dplyr verbs into SQL and execute on the table without pulling all rows from the table

In [12]:
encounters = return_table("encounters")
encounters

# Source:   table<dbo.encounters> [?? x 4]
# Database: Microsoft SQL Server 13.00.5830[@ESMPMDBPR4/CAMP_PMCoE_Projection]
   osler_id                             encounter_id encou~1 encounter_date     
   <chr>                                       <int> <chr>   <dttm>             
 1 5303550b-8ed2-42fd-885a-d32b308b05f3        17938 Office~ 2015-10-10 00:00:00
 2 5303550b-8ed2-42fd-885a-d32b308b05f3       142706 Office~ 2016-01-24 00:00:00
 3 5303550b-8ed2-42fd-885a-d32b308b05f3       571465 Office~ 2017-03-19 00:00:00
 4 5303550b-8ed2-42fd-885a-d32b308b05f3       432470 Office~ 2016-10-22 00:00:00
 5 5303550b-8ed2-42fd-885a-d32b308b05f3       410795 Office~ 2016-10-01 00:00:00
 6 5303550b-8ed2-42fd-885a-d32b308b05f3       631974 Office~ 2017-05-08 00:00:00
 7 5303550b-8ed2-42fd-885a-d32b308b05f3       483944 Office~ 2016-12-10 00:00:00
 8 5303550b-8ed2-42fd-885a-d32b308b05f3       411794 Office~ 2016-10-03 00:00:00
 9 5303550b-8ed2-42fd-885a-d32b308b05f3       324374 Office~ 2016-07

For example, we can do `"SELECT encounter_type, count() as CT from encounters group by encounter_type ORDER BY COUNT()"` without explicitly writing the SQL query

In [13]:
dplyr::count(encounters, encounter_type, sort=T)

# Source:     SQL [?? x 2]
# Database:   Microsoft SQL Server 13.00.5830[@ESMPMDBPR4/CAMP_PMCoE_Projection]
# Ordered by: desc(n)
   encounter_type          n
   <chr>               <int>
 1 Office Visit       431080
 2 Appointment         97718
 3 Hospital Encounter  86789
 4 Visit Encounter     62740
 5 Clinical Support    21626
 6 Procedure visit     18256
 7 Results Only         8895
 8 Orders Only          8635
 9 Anti-coag visit      7284
10 Provider Procedure   2987
# ... with more rows
# i Use `print(n = ...)` to see more rows

We can of course pull the entire table locally, if we desire, using `dplyr::collect()`

In [14]:
encounters_local = encounters %>% dplyr::collect()
print(encounters_local %>% head(10))

# A tibble: 10 x 4
   osler_id                             encounter_id encou~1 encounter_date     
   <chr>                                       <int> <chr>   <dttm>             
 1 5303550b-8ed2-42fd-885a-d32b308b05f3        17938 Office~ 2015-10-10 00:00:00
 2 5303550b-8ed2-42fd-885a-d32b308b05f3       142706 Office~ 2016-01-24 00:00:00
 3 5303550b-8ed2-42fd-885a-d32b308b05f3       571465 Office~ 2017-03-19 00:00:00
 4 5303550b-8ed2-42fd-885a-d32b308b05f3       432470 Office~ 2016-10-22 00:00:00
 5 5303550b-8ed2-42fd-885a-d32b308b05f3       410795 Office~ 2016-10-01 00:00:00
 6 5303550b-8ed2-42fd-885a-d32b308b05f3       631974 Office~ 2017-05-08 00:00:00
 7 5303550b-8ed2-42fd-885a-d32b308b05f3       483944 Office~ 2016-12-10 00:00:00
 8 5303550b-8ed2-42fd-885a-d32b308b05f3       411794 Office~ 2016-10-03 00:00:00
 9 5303550b-8ed2-42fd-885a-d32b308b05f3       324374 Office~ 2016-07-11 00:00:00
10 5303550b-8ed2-42fd-885a-d32b308b05f3       705948 Office~ 2017-07-19 00:00:00
# ... wit

#### Use `query_db()` to submit any sql query directly to the database; by default it returns a lazy tbl

In [16]:
qry <- "SELECT encounter_type, ct = COUNT(*) FROM encounters GROUP BY encounter_type"
query_db(qry) %>% dplyr::arrange(desc(ct))

# Source:     SQL [?? x 2]
# Database:   Microsoft SQL Server 13.00.5830[@ESMPMDBPR4/CAMP_PMCoE_Projection]
# Ordered by: desc(ct)
   encounter_type         ct
   <chr>               <int>
 1 Office Visit       431080
 2 Appointment         97718
 3 Hospital Encounter  86789
 4 Visit Encounter     62740
 5 Clinical Support    21626
 6 Procedure visit     18256
 7 Results Only         8895
 8 Orders Only          8635
 9 Anti-coag visit      7284
10 Provider Procedure   2987
# ... with more rows
# i Use `print(n = ...)` to see more rows

#### Use gen_random_table() to get subset of the identifying variables for a table; the function creates a table on the db, and returns the name of that temp table

In [18]:
random_px = gen_random_table("patients",idvars="osler_id", engine=default_engine)
cat("Name of the newly created table is ", random_px,"\n")

Name of the newly created table is  #qw1whqw431npwtjs 


### You can feed this table name, just like any other table name, to the `return_table()` function

In [20]:
return_table(random_px, engine=default_engine)

# Source:   table<dbo.#qw1whqw431npwtjs> [?? x 1]
# Database: Microsoft SQL Server 13.00.5830[@ESMPMDBPR4/CAMP_PMCoE_Projection]
   osler_id                            
   <chr>                               
 1 02d6224e-f7cb-4b5d-9868-952360cdcaa9
 2 02f55980-ab50-41aa-812d-0c642848fa45
 3 044dc300-c300-4b51-b345-48475c895187
 4 0edef1fb-02ea-473d-b1bb-6bedc77c26b0
 5 1431a676-d36b-4653-8c82-db96bb157f13
 6 1d9d520e-b137-4182-92e4-3ac663791617
 7 2407e141-1a56-483b-931a-3380a6e5d1c7
 8 27137853-1129-4e99-b42c-0d601567b779
 9 27b0ed87-12b2-4d28-9da3-211de651cd9f
10 365dbbb0-3e6b-4479-8b99-6cc9e6f95620
# ... with more rows
# i Use `print(n = ...)` to see more rows

### This table of random ids could be used in simple joins, for example, below we get all the rows from the `vitals_weight` table for the osler ids in the `random_px` table.

In [21]:
return_table("vitals_weight") %>% 
    dplyr::inner_join(return_table(random_px), by="osler_id")

# Source:   SQL [?? x 7]
# Database: Microsoft SQL Server 13.00.5830[@ESMPMDBPR4/CAMP_PMCoE_Projection]
   osler_id       encou~1 encou~2 admission_date      discharge_date weight
   <chr>            <int> <chr>   <dttm>              <dttm>         <chr> 
 1 e490d206-db28~  113734 Office~ 2017-01-01 00:00:00 NA             35.1  
 2 e490d206-db28~  216667 Office~ 2016-11-30 00:00:00 NA             34.93 
 3 e490d206-db28~  171900 Office~ 2016-10-24 00:00:00 NA             36.03 
 4 e490d206-db28~  439056 Office~ 2017-07-02 00:00:00 NA             36.8  
 5 e490d206-db28~  394328 Office~ 2017-05-20 00:00:00 NA             35.29 
 6 e490d206-db28~  179107 Office~ 2016-12-24 00:00:00 NA             32.74 
 7 e490d206-db28~  108444 Proced~ 2016-08-23 00:00:00 NA             34.7  
 8 e490d206-db28~   45749 Office~ 2016-08-23 00:00:00 NA             34.7  
 9 e490d206-db28~   10863 Office~ 2016-07-31 00:00:00 NA             34.47 
10 e490d206-db28~  100630 Office~ 2016-08-07 00:00:00 NA    